In [34]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re

In [35]:
def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '', raw_html)
  regex = re.compile('[^A-Za-zА-Яа-я]')
  cleantext = regex.sub(' ', cleantext)
  return cleantext

In [36]:
def isvotes(tag):
    return tag.has_attr('class') and tag.has_attr('onclick') and not tag.has_attr('href') and not tag.has_attr('type')

In [50]:
def document(id):
    page = requests.get('https://habr.com/ru/post/' + str(id) + '/')
    soup = BeautifulSoup(page.text, 'html5lib')
    info = []
    if soup.find("span", class_= "post__title-text"):
        #title
        info.append(soup.find("span", class_= "post__title-text").text)
        #text
        text = soup.find("div", class_="post__text post__text-html").text
        info.append(cleanhtml(text))
        #year
        info.append(int(soup.find("span", class_= "post__time").text.split()[2]))
        #tags
        tagsshit = soup.find_all("a", class_="inline-list__item-link hub-link")
        tags = []
        for i in range(len(tagsshit)):
            tags.append(tagsshit[i].text.split())
        info.append(tags)
        #votes
        a = soup.find(isvotes).text
        if a[0] == '–':
            b = int(a[1:])
            info.append(-b)
        else:
            info.append(int(a))
        #views
        a = soup.find("div", class_="post-stats__views").text.strip()
        if 'k' in a:
            views = float(a[:-1].replace(',', '.')) * 1000
        else:
            views = int(a)
        info.append(views)
        #bookmarks
        info.append(int(soup.find("span", class_="bookmark__counter js-favs_count").text))
        #comments
        comment1 = soup.find_all("div", class_="comment__message")
        comments = []
        for i in range(len(comment1)):
            comments.append(comment1[i].text.split())
        info.append(comments)
    else:
        info.append('NaN')
    print(id)
    return info

In [ ]:
from multiprocessing import Pool
with Pool(100) as p:
    a = p.map(document, np.arange(206668, 489000))

In [51]:
b = document(207456)

207456


In [45]:
b

['Стандарт Miracast — старые протоколы в новой обёртке',
 'Не так давно  начиная с JellyBean      Google добавила в Android поддержку технологии Miracast   Практическому исследованию этой технологии методами reverse engineering и посвящена статья   Что такое Miracast в двух словах  Это очередное детище Wi Fi альянса   стандарт для передачи мультимедийного контента по сети Wi Fi в peer to peer режиме  Для пользователя это означает прежде всего то  что для соединения с телевизором  к примеру  ему не понадобится Wi Fi маршрутизатор  Два устройства по задумке альянса должны связываться друг с другом напрямую  Это обеспечивается использованием стандарта Wi Fi Direct за авторством той же организации  Иными словами  новый стандарт решает задачи очень похожие на AirPlay от Apple  WiDi от Intel  или старое доброе DLNA   Зачем было городить огород   спросите вы  Почему было не воспользоваться уже существующим решением  Тут мне будет трудно ответить  Понятно  что лицензировать решения от прямых к

In [53]:
df = pd.DataFrame([b], columns=['title', 'text', 'year', 'tags', 'votes', 'views', 'bookmarks', 'comments'])
df['tags'][0]

[['Сетевые', 'технологии']]

In [7]:
a = pd.read_csv('habr(2).csv')
a

,Unnamed: 0,title,text,year,tags,votes,views,bookmarks,comments
0,0,Бенчмарк HTTP-серверов (С/C++) в FreeBSD,Проведено сравнение производительности ядер ...,2013,Высокая производительностьC++,34,30800.0,143,В ходе разговора с авторами nginx они нескольк...


In [32]:
tags = []
for i in a['tags']:
    tags.append(i.split())

In [ ]:
df.to_csv('habr.csv')
from google.colab import drive
drive.mount('/content/drive')

from google.colab import files
files.download('habr.csv')`